In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from subprocess import check_output
print(check_output(["ls", "archive"]).decode("utf8"))

coin_Aave.csv
coin_BinanceCoin.csv
coin_Bitcoin.csv
coin_Cardano.csv
coin_ChainLink.csv
coin_Cosmos.csv
coin_CryptocomCoin.csv
coin_Dogecoin.csv
coin_EOS.csv
coin_Ethereum.csv
coin_Iota.csv
coin_Litecoin.csv
coin_Monero.csv
coin_NEM.csv
coin_Polkadot.csv
coin_Solana.csv
coin_Stellar.csv
coin_Tether.csv
coin_Tron.csv
coin_USDCoin.csv
coin_Uniswap.csv
coin_WrappedBitcoin.csv
coin_XRP.csv



In [3]:
df = pd.read_csv("archive/coin_Bitcoin.csv", parse_dates=['Date'])
df.head()

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Bitcoin,BTC,2013-04-29 23:59:59,147.488007,134.000000,134.444000,144.539993,0.0,1.603769e+09
1,2,Bitcoin,BTC,2013-04-30 23:59:59,146.929993,134.050003,144.000000,139.000000,0.0,1.542813e+09
2,3,Bitcoin,BTC,2013-05-01 23:59:59,139.889999,107.720001,139.000000,116.989998,0.0,1.298955e+09
3,4,Bitcoin,BTC,2013-05-02 23:59:59,125.599998,92.281898,116.379997,105.209999,0.0,1.168517e+09
4,5,Bitcoin,BTC,2013-05-03 23:59:59,108.127998,79.099998,106.250000,97.750000,0.0,1.085995e+09


In [7]:
df.columns

Index(['SNo', 'Name', 'Symbol', 'Date', 'High', 'Low', 'Open', 'Close',
       'Volume', 'Marketcap', 'Date_mpl'],
      dtype='object')

In [10]:
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
import os 

RootDir = "archive/"
History = 60

def read_data ():
    coin_no = 0
    for name in os.listdir(RootDir):
        coin_no += 1
    
    max_length, min_length = 0, 1000000
    for name in os.listdir(RootDir):
        df = pd.read_csv(RootDir + "/" + name, parse_dates=['Date'])
        length = df.shape[0]
        if max_length < length:
            max_length = length
        if min_length > length:
            min_length = length
    
    data = np.zeros ((coin_no, max_length))
    lengths = np.zeros(coin_no, dtype = int)
    i = 0
    for name in os.listdir(RootDir):
        short_name = name[5:-4]
        df = pd.read_csv(RootDir + "/" + name, parse_dates=['Date'])
        length = df.shape[0]
        lengths[i] = length
        print (i, short_name, length)
        data[i, 0:length] = df['Close'].values  # We only keep the closing price as a sequence!
        i += 1
    
    return coin_no, lengths, data

coin_no, lengths, data = read_data ()
print ("Got", coin_no, "coins.")

0 NEM 2288
1 EOS 1466
2 Monero 2602
3 Polkadot 320
4 USDCoin 1002
5 Uniswap 292
6 BinanceCoin 1442
7 Iota 1484
8 Aave 275
9 Solana 452
10 Bitcoin 2991
11 Cardano 1374
12 Tether 2318
13 Cosmos 845
14 ChainLink 1385
15 Litecoin 2991
16 XRP 2893
17 Ethereum 2160
18 Tron 1392
19 Stellar 2527
20 CryptocomCoin 935
21 Dogecoin 2760
22 WrappedBitcoin 888
Got 23 coins.


In [11]:
def scale_data (data, lengths):
    coin_no = data.shape[0]
    shift = np.zeros (coin_no)
    factor = np.zeros (coin_no)
    for i in range (coin_no):
        max_val = data[i,:lengths[i]].max()
        min_val = data[i, :lengths[i]].min()
        shift[i] = min_val
        factor[i] = max_val - min_val
        data[i,0:lengths[i]] = (data[i,0:lengths[i]]-shift[i])/factor[i]
    return (shift, factor)
    
shift, factor = scale_data (data, lengths)

In [12]:
def create_sequences (data, lengths, start, end):
    x = []
    y = []
    for i in range (start, end):   # Go only over the specified coins
        for j in range(History, lengths[i]):
            x.append(data[i, j-History:j])
            y.append(data[i, j])
    return np.array(x)[:, :, np.newaxis], np.array(y)

x_train, y_train = create_sequences(data, lengths, 0, 18)
print ("Got", y_train.shape[0], "training sequenes.")
x_val, y_val = create_sequences(data, lengths, 18, 22)
print ("Got", y_val.shape[0], "validation sequenes.")
x_test, y_test = create_sequences(data, lengths, 22, 23)
print ("Got", y_test.shape[0], "test sequenes.")

Got 27500 training sequenes.
Got 7374 validation sequenes.
Got 828 test sequenes.


In [ ]:
hist = model.fit(x_train, y_train, validation_data = (x_val, y_val), 
              batch_size=32, epochs=5) 

def show_stats (hist):
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title("Model loss")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend(["Training loss","Validation loss"])
    plt.show()

show_stats (hist)